In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats

#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import selection_tests

In [19]:
class DiscreteOutcomes(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(DiscreteOutcomes,self).__init__(*args,**kwargs)

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        present_bias = x[:,0]
        future_bias = x[:,1]
        likelihood = (params[0]*present_bias*(1-future_bias) +
                      params[1]*(1-present_bias)*future_bias + 
                      params[2]*(future_bias)*(present_bias) +
                      params[3]*(1-present_bias)*(1-future_bias))
        return np.log(likelihood)

def return_params(xn):
    xn = np.array(xn)
    present_bias = xn[:,0]
    future_bias = xn[:,1]
    return np.array([(present_bias*(1-future_bias)).mean(),
            ((1-present_bias)*future_bias).mean(), 
            ((future_bias)*(present_bias)).mean(),
            ((1-present_bias)*(1-future_bias)).mean()])


data = pd.read_csv('biases.csv',index_col=0)
params = np.array([19/64,19/64,18/64,8/64])
print(return_params(data[['present_bias','future_bias']]))
model1_deriv = DiscreteOutcomes(data['client'],data[['present_bias','future_bias']])
ll1 = model1_deriv.loglikeobs(params)
grad1 =  model1_deriv.score_obs(params)
hess1 = model1_deriv.hessian(params)
print(ll1.shape)

[0.75       0.15       0.08333333 0.01666667]
(60,)


In [22]:
def setup_test(yn,xn,return_model=False):
    # model 1 grad, etc.
    params1 = return_params(xn)
    model1_deriv = DiscreteOutcomes(yn,xn)
    ll1 = model1_deriv.loglikeobs(params1)
    grad1 =  model1_deriv.score_obs(params1)    
    hess1 = model1_deriv.hessian(params1)

    #19/64, 19/64, 18/64, 8/64
    params2 = np.array([19/64,19/64,18/64,8/64])
    model2_deriv = DiscreteOutcomes(yn,xn)
    ll2 = model2_deriv.loglikeobs(params2)
    grad2 =  model2_deriv.score_obs(params2)    
    hess2 = model2_deriv.hessian(params2)
    
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

#setup_test(data['client'],data[['present_bias','future_bias']])

(array([-2.48490665, -0.28768207, -0.28768207, -0.28768207, -2.48490665,
        -0.28768207, -0.28768207, -4.09434456, -0.28768207, -0.28768207,
        -0.28768207, -0.28768207, -0.28768207, -0.28768207, -0.28768207,
        -0.28768207, -0.28768207, -1.89711998, -0.28768207, -1.89711998,
        -1.89711998, -0.28768207, -0.28768207, -0.28768207, -0.28768207,
        -2.48490665, -0.28768207, -1.89711998, -0.28768207, -0.28768207,
        -0.28768207, -2.48490665, -0.28768207, -0.28768207, -0.28768207,
        -0.28768207, -0.28768207, -0.28768207, -0.28768207, -1.89711998,
        -0.28768207, -0.28768207, -0.28768207, -1.89711998, -0.28768207,
        -2.48490665, -1.89711998, -0.28768207, -0.28768207, -0.28768207,
        -1.89711998, -0.28768207, -0.28768207, -0.28768207, -0.28768207,
        -0.28768207, -0.28768207, -1.89711998, -0.28768207, -0.28768207]),
 array([[ 0.        ,  0.        , 12.        ,  0.        ],
        [ 1.33333333,  0.        ,  0.        ,  0.        ]

In [23]:
print(selection_tests.test_table(data['client'],data[['present_bias','future_bias']],setup_test))

V ----
(5.225103659017396e-07-1.1249656330721176e-22j)
----
---- bootstrap: llr, omega ----
(27.463329382477063-1.1249656330721176e-22j) (6.460009235190213+0j)
----
regular: test, llr, omega ----
4.251283219590945 27.463328859966698 0.8339836061412603
---- 
\begin{center}
\begin{tabular}{ccc}
\toprule
\textbf{Version} & \textbf{Result} & \textbf{95 \% CI} \\ \midrule
Bootstrap & H1 & [2.147, 7.456] \\
Shi (2015) & H0 & [-4.869, 13.371] \\
Classical & H1 & [2.292, 6.210] \\
\bottomrule
\end{tabular}
\end{center}
None


../selection_tests.py:118: ComplexWarning: Casting complex values to real discards the imaginary part
  print('Bootstrap & H%s & [%.3f, %.3f] \\\\'%(result_boot,cv_lower,cv_upper))
../selection_tests.py:119: ComplexWarning: Casting complex values to real discards the imaginary part
  print('Shi (2015) & H%s & [%.3f, %.3f] \\\\'%(result_shi, stat_shi- cv_shi,stat_shi+ cv_shi))
